In [1]:
from dataset import tweetProccess
import nltk, re

'''gets the data'''
dataHandler = tweetProccess.FileTreat()
content = dataHandler.getTweets()


'''Use to download nltk content'''
#nltk.download()

'''defining a function to remove numbers'''
def tokenize(text):
    tokenized = []
    for word in nltk.word_tokenize(text):
        if re.search('[a-zA-Z]', text):
            tokenized.append(word.lower())
            
    return tokenized
    

In [2]:
'''create a matrix of words'''
from sklearn.feature_extraction.text import CountVectorizer

no_features = 2000

# LDA can only use raw term counts for LDA because it is a probabilistic graphical model
tf_vectorizer = CountVectorizer(tokenizer=tokenize, max_df=0.95, min_df=2, max_features=no_features, stop_words='english')
tf = tf_vectorizer.fit_transform(content)
tf_feature_names = tf_vectorizer.get_feature_names()

print(tf.shape)

(178434, 2000)


In [3]:
print(len(content))
print(tf)
#print(tf[62457])
print(len(tf_feature_names))
#pq n_samples esta sempre zerado?

178434
  (0, 821)	1
  (0, 70)	2
  (0, 74)	2
  (0, 1421)	1
  (1, 752)	1
  (1, 1812)	1
  (1, 931)	1
  (1, 904)	1
  (2, 12)	1
  (2, 1579)	1
  (2, 1588)	1
  (2, 93)	1
  (2, 184)	1
  (2, 1426)	1
  (2, 675)	1
  (2, 451)	2
  (2, 8)	1
  (2, 1629)	1
  (2, 347)	1
  (2, 713)	1
  (2, 821)	1
  (2, 70)	1
  (3, 1044)	1
  (3, 113)	1
  (3, 1137)	1
  :	:
  (178431, 1692)	1
  (178431, 1764)	1
  (178431, 14)	1
  (178431, 937)	1
  (178431, 1132)	1
  (178431, 852)	1
  (178431, 1027)	2
  (178431, 12)	2
  (178431, 70)	1
  (178431, 74)	1
  (178431, 1421)	1
  (178432, 1413)	1
  (178432, 1226)	1
  (178432, 1681)	1
  (178432, 225)	1
  (178432, 1132)	1
  (178432, 827)	1
  (178432, 12)	5
  (178433, 92)	1
  (178433, 1652)	1
  (178433, 904)	1
  (178433, 821)	1
  (178433, 70)	2
  (178433, 74)	1
  (178433, 1421)	1
2000


In [ ]:
'''searches for the no_topics topics'''

from sklearn.decomposition import LatentDirichletAllocation
no_topics = 10

lda = LatentDirichletAllocation(n_topics=no_topics, max_iter=5, learning_method='online', learning_offset=50.,random_state=0).fit(tf)

/home/yuri/anaconda3/lib/python3.5/site-packages/sklearn/decomposition/online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


In [7]:
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic %d:" % (topic_idx))
        print (" ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

no_top_words = 10
display_topics(lda, tf_feature_names, no_top_words)

Topic 0:
! @ ; & ) amp ( , thank school
Topic 1:
, . life need ’ happy today got ” “
Topic 2:
. , dont like know ... feel love people think
Topic 3:
@ . help new - world ? great video real
Topic 4:
# : https wearereadyharry loved hope selfharm depression thinspo program
Topic 5:
? just , u right anymore na let ill wan
Topic 6:
: @ rt https http line cross cr0sstheline cuttingquote important
Topic 7:
im . fucking hate fuck ... ive stop going shit
Topic 8:
, . anxiety depression mental god … music depressed feelings
Topic 9:
. want just , wish sad time die friends fat


In [8]:
#from sklearn.metrics.pairwise import cosine_similarity
#dist = 1 - cosine_similarity(tf)

#print(dist)